### 1.0 Libraries and set-up

In [2]:
import ee
import os
import math
import geemap
import json
from shapely.geometry import shape
import geopandas as gpd

In [3]:
ee.Authenticate()

True

In [4]:
ee.Initialize()

In [5]:
project_dir = 'projects/alpod-412314/assets/' 

region = 'YKD'
region_path = f'{region}_weekly'
folder = project_dir + region_path

In [6]:
image_list = ee.data.listImages(folder)
image_list = image_list.get('images')

In [7]:
lake_extractions_list = ee.data.listAssets({'parent':  project_dir + 'Lake_extractions/'})
#print(lake_extractions_list)

In [8]:
lakes_path = f'{project_dir}Lake_extractions/{region}_extraction'
lake_polygons = ee.FeatureCollection(lakes_path)

In [9]:
print(lake_polygons.getInfo)

<bound method Collection.getInfo of <ee.featurecollection.FeatureCollection object at 0x15bcb1520>>


### 2.0 Images for target time period and region

In [10]:
target_years = list(range(2016, 2024))
target_years = [str(year) for year in target_years]
print(target_years)

# June
target_weeks = list(range(22, 27))
#August 
#target_weeks = list(range(31, 36))
# September
#target_weeks = list(range(35, 41))


target_weeks = [str(week) for week in target_weeks]
print(target_weeks)

max_times = len(target_years) * len(target_weeks)
print(max_times)

['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
['22', '23', '24', '25', '26']
40


In [11]:
target_imgs = []
target_polygons = []

for image in image_list:
    img_id = image['id']
    year = img_id.split('_')[-2]
    week = img_id.split('_')[-1]

    if week in target_weeks and year in target_years:
        img_ee = ee.Image(img_id)
        img_properties = img_ee.getInfo()['properties']
        img_coords = img_properties['system:footprint']['coordinates']
        polygon_ee = ee.Geometry.Polygon(img_coords)
        
        target_imgs.append(img_ee)
        target_polygons.append(polygon_ee)

print(len(target_imgs))

319


### 3.0 Export the ROI foot-print

In [12]:
p1 = target_polygons[0]
p2 = target_polygons[1]

combined_footprint = p1.union(p2)

for i in range(2, len(target_polygons)):
    p = target_polygons[i]
    combined_footprint = combined_footprint.union(p)

# combined_footprint_info = combined_footprint.getInfo()
# geojson_footprint = json.dumps(combined_footprint_info)
# geometry = shape(json.loads(geojson_footprint))
# roi_gdf = gpd.GeoDataFrame([{'geometry': geometry, 'roi': f'{region}'}], crs="EPSG:4326")
# os.chdir('/Users/jmaze/Documents/projects/altimetry_lakes_v3/')
# roi_gdf.to_file(f'./data/ew_rois/{region}.shp')

### 4.0 Determine the observation counts for each pixel

In [40]:
print(lake_polygons.getInfo)
t = ee.Feature(lake_polygons.first())
print(t.toDictionary().getInfo())

<bound method Collection.getInfo of <ee.featurecollection.FeatureCollection object at 0x15bcb1520>>
{'area_km2': 4.8e-05, 'chain_id': 50441, 'lake_id': 'YKDaaaniR', 'n_lakes': 1, 'perim_km': 0.03, 'region': 'YKD'}


In [58]:
test_num = 130
rando_img = target_imgs[test_num]

#print(rando_img.getInfo())
temp = ee.Image().byte()

buffered_lakes_rast = temp.paint(
    featureCollection=lake_polygons,
    color=1
)
buffered_lakes_rast = buffered_lakes_rast.unmask(0)

#rando_img_obs = rando_img.where(rando_img.eq(0).And(rando_img.eq(1)), 1).unmask(0)

conditional_image = buffered_lakes_rast.expression(
    '((buffered_lakes_rast == 1) && (rando_img == 1)) ? 100 : '  # Condition 1: both images equal 1
    '((buffered_lakes_rast == 1) && (rando_img == 0)) ? 50 : 0', # Condition 2: buffered_lakes_rast equals 1, rando_img equals 0
    {
        'buffered_lakes_rast': buffered_lakes_rast, 
        'rando_img': rando_img,
    }
)

conditional_image = conditional_image.clip(target_polygons[test_num])

water_viz = {
    'bands': ['constant'],
    'min': 0,
    'max': 100,
    'palette': ['red', 'brown', 'blue']
}

buffered_lakes_viz = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'pink']
}

print(conditional_image.getInfo())

Map = geemap.Map(center=(65, -135), zoom=4)
Map.addLayer(conditional_image, water_viz, 'test')
#Map.addLayer(buffered_lakes_rast, buffered_lakes_viz, 'hmm')
Map


{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 100}, 'dimensions': [4, 3], 'origin': [-164, 59], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[-162.9256914239813, 61.32165149933874], [-163.18881581112868, 61.32165138295492], [-163.18880979106663, 60.294337706358654], [-163.05715976763253, 60.2943378112966], [-162.85995711888265, 60.29428907198459], [-162.59702019226236, 60.2942891516568], [-162.39981761085133, 60.294337803721575], [-162.2026149633654, 60.29428907928763], [-162.00541231592624, 60.29433782970371], [-161.80820965330386, 60.294289088168846], [-161.54527278022567, 60.29428911633803], [-161.28233580583495, 60.29428914769383], [-161.0849515531911, 60.294337687002965], [-161.08494560654447, 61.321651414516175], [-161.47953852761117, 61.321651526300926], [-161.7424752968401, 61.32165150834502], [-162.1368806967916, 61.3216

Map(center=[65, -135], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

### 4.1 Visualize the observation counts

In [ ]:
viz_obs = {
    'bands': ['constant'],
    'min': 0,
    'max': max_times,
    'palette': ['ff0000', 'ffff00', '00ff00']
}

Map = geemap.Map(center=(65, -135), zoom=4)
Map.addLayer(lake_polygons, {}, '.')
Map.addLayer(pixel_observations, viz_obs, f'{max_times}')
Map

### 5.0 Calculate the number of water occurances for each pixel

In [21]:
target_collection = ee.ImageCollection(target_imgs)
wtr_sum_img = target_collection.select('water_occurance').sum()
wtr_sum_img = wtr_sum_img.clip(combined_footprint)

### 6.0 Dived each pixels water occurance by total observations

In [21]:
wtr_proportion = wtr_sum_img.select('water_occurance').divide(pixel_observations.select('constant'))

### 6.1 Visualize the water occurance proportions

In [ ]:
viz_prop = {
    'bands': ['water_occurance'],
    'min': 0,
    'max': 1,
    'palette': ['ff0000', '00ff00', '0000ff']
}

Map = geemap.Map(center=(65, -135), zoom=4)
Map.addLayer(wtr_proportion, viz_prop, '.')
Map



### 7.0 Binary image based on number of occurances

In [12]:
time_fractions = [1/5, 1/6, 1/7, 1/10, 1/15]
#print(time_fractions)
water_thresholds = [time_fraction * max_times for time_fraction in time_fractions]
print(water_thresholds)

binary_images = {}
for threshold in water_thresholds:
    threshold_round = math.floor(threshold)
    wtr_img_binary = wtr_sum_img.gt(threshold)
    binary_images[threshold_round] = wtr_img_binary

print(binary_images)

[8.0, 6.666666666666666, 5.7142857142857135, 4.0, 2.6666666666666665]
{8: <ee.image.Image object at 0x165929460>, 6: <ee.image.Image object at 0x165a1fb90>, 5: <ee.image.Image object at 0x165a1d9a0>, 4: <ee.image.Image object at 0x165a1d8b0>, 2: <ee.image.Image object at 0x165a1ffb0>}


### 6.0 Visualize results

In [13]:
viz1 = {
    'bands': ['water_occurance'],
    'min': 0,
    'max': 1,
    'palette': ['ff0000', '00ff00', '0000ff']
}

Map = geemap.Map(center=(65, -135), zoom=4)

occurance_cnts = binary_images.keys()
print(occurance_cnts)

for cnt in occurance_cnts:
    Map.addLayer(binary_images[cnt], viz1, f'{region}, {cnt}, {target_weeks[0]}')


#Map.addLayer(combined_footprint, {}, 'roi_bounds')
Map


dict_keys([8, 6, 5, 4, 2])


Map(center=[65, -135], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [18]:
viz2 = {
    'bands': ['water_occurance'],
    'min': 0,
    'max': 4,
    'palette': ['ff0000', '00ff00', '0000ff']
}

Map2 = geemap.Map(center=(65,-135), zoom=4)
Map2.addLayer(wtr_sum_img , viz2, f'Occurence sums')
Map2

Map(center=[65, -135], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [ ]:


### 7.1 Export the .GeoTiff to Drive

In [13]:
for cnt in occurance_cnts:

    img = binary_images[cnt]

    export_task = ee.batch.Export.image.toDrive(
        image=img,
        description=f'{region}_count{cnt}_years{target_years[0]}-{target_years[-1]}_weeks{target_weeks[0]}-{target_weeks[-1]}',
        folder='ewebb_s2',
        fileNamePrefix=f'{region}_count{cnt}_years{target_years[0]}-{target_years[-1]}_weeks{target_weeks[0]}-{target_weeks[-1]}',
        region=combined_footprint,
        scale=10,
        fileFormat='GeoTIFF',
        maxPixels=1e13
    )
    
    export_task.start()

### X.2 Export the .shp files to Drive??

### * Scrap workspace

In [ ]:
# obs_masks = []
# for polygon in target_polygons:
#     obs_mask = ee.Image.constant(1).clip(polygon)
#     #obs_mask = obs_mask.set(1)
#     obs_masks.append(obs_mask)
    
# obs_collection = ee.ImageCollection(obs_masks)

# pixel_observations = obs_collection.select('constant').sum()
# #print(pixel_observations)

In [20]:
rando_img = target_imgs[58]
viz2 = {
    'bands': ['water_occurance'],
    'min': 0,
    'max': 1,
    'palette': ['white', 'blue']
}

Map2 = geemap.Map(center=(65,-135), zoom=4)
Map2.addLayer(rando_img , viz2, f'idk')
Map2.addLayer(lake_polygons, {}, 'wtf')
Map2
#print(rando_img.getInfo)

Map(center=[65, -135], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…